# Predikcija održavanja sinusnog ritma posle godinu dana
Zadatak:
- Pronaći atribute značajne za predikciju uspešnosti održavanja sinusnog srčanog ritma godinu dana nakon obavljene elektrokonverzije u odnosu na kliničku sliku pacijenta, druge indikacije i dodeljenu terapiju
- Konstruisati bajesovsku mrežu koja uključuje relevantne atribute i vrši predikciju uspešnosti održavanja sinusnog ritma godinu dana nakon elektrokonverzije
- Testirati druge metode istraživanja podataka (neke od: SVM, neuronske mreže, stabla odlučivanja, logistička regresija) i uporediti sa uspešnosti predikcije korišćenjem bajesovske mreže

In [74]:
import re

import numpy as np
import pandas as pd

In [75]:
# kolonu H zadrzavamo jer je koristimo pri konverziji kolone J
data = pd.read_excel('././elektrokonverzija.xlsx', usecols='A,C:E,H,I:AH,AJ:BB,BE:BI,CE:CT,CW', nrows=147)

/Users/sterlu/opt/miniconda3/envs/ml/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Menjamo nazive kolona - izbacujemo dodatne podatke koji objašnjavaju vrednosti.

In [76]:
postojece_kolone = []
def preimenuj(kol):
    index = min([a for a in [99999, kol.find('  '), kol.find('0 - '), kol.find('1 - ')] if a > -1])
    if index < 99999:
        kol = kol[0:index].strip()
    print(kol, kol in postojece_kolone)
    if kol in postojece_kolone:
        kol += '.1'
    postojece_kolone.append(kol)
    return kol
data.columns = data.columns.map(preimenuj)
data.columns

Broj False
godine starosti False
stariji od 65 False
Pol False
Datum elektrokonverzije False
indikacija: perzistentna False
ukupno trajanje AF (kada je prvi put dijagnostikovana) u mesecima False
trajanje ove epizode AF u mesecima False
frekvenca komora na prijemnom EKG-u False
Blok grane False
dimenzija LPK False
EDD LK False
ESD LK False
EF LK False
MR False
TR False
HTA False
DM False
HLP False
HOBP False
pušenje False
Hereditet za KVB False
ishemijska bolest srca False
HF False
NYHA klasa (ukoliko ima HF) False
Prethodni TIA/CVI False
HBI False
CHADS-Vasc False
poremećaj f-je štitaste žlezde False
dosadašnje elektrokonverzije False
SCD (Structural heart disease) False
Beta blokatori False
ACE inhibitori False
ARB False
Furosemid (ne - 0) False
Drugi diuretik False
Spironolakton False
Ca antagonisti False
Amiodaron False
Propafen False
Flekainid False
Sotalol False
Digoksin False
ASA False
OAKT False
vrednost INR 2-3 False
Plavix False
Statin False
Implantiran CIED False
Vreme u mes

Index(['Broj', 'godine starosti', 'stariji od 65', 'Pol',
       'Datum elektrokonverzije', 'indikacija: perzistentna',
       'ukupno trajanje AF (kada je prvi put dijagnostikovana) u mesecima',
       'trajanje ove epizode AF u mesecima',
       'frekvenca komora na prijemnom EKG-u', 'Blok grane', 'dimenzija LPK',
       'EDD LK', 'ESD LK', 'EF LK', 'MR', 'TR', 'HTA', 'DM', 'HLP', 'HOBP',
       'pušenje', 'Hereditet za KVB', 'ishemijska bolest srca', 'HF',
       'NYHA klasa (ukoliko ima HF)', 'Prethodni TIA/CVI', 'HBI', 'CHADS-Vasc',
       'poremećaj f-je štitaste žlezde', 'dosadašnje elektrokonverzije',
       'SCD (Structural heart disease)', 'Beta blokatori', 'ACE inhibitori',
       'ARB', 'Furosemid (ne - 0)', 'Drugi diuretik', 'Spironolakton',
       'Ca antagonisti', 'Amiodaron', 'Propafen', 'Flekainid', 'Sotalol',
       'Digoksin', 'ASA', 'OAKT', 'vrednost INR 2-3', 'Plavix', 'Statin',
       'Implantiran CIED',
       'Vreme u mesecima od ugradnje aparata do elektrokonve

In [77]:
print('Ukupno pacijenata:', data.shape[0])

relevant_data = data[data['Elektrokonverzija'] < 3]
print('Pacijenata sa uspesnom el. konv. (relevantni podaci):', relevant_data.shape)
print('Pacijenata sa odrzanim ritmom posle god. (pozitivan ishod):', relevant_data[relevant_data['Održavanje sinusnog ritma nakon godinu dana'] == 1].shape)

# relevant_data_2 = relevant_data[pd.isna(relevant_data['Broj dana do javljanja AF'])]
# print('Pacijenata sa uspesnom el. konv. i posle mesec dana', relevant_data_2.shape[0])

Ukupno pacijenata: 147
Pacijenata sa uspesnom el. konv. (relevantni podaci): (130, 72)
Pacijenata sa odrzanim ritmom posle god. (pozitivan ishod): (50, 72)


In [78]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# relevant_data

Formatiramo podatke mešovitih formata. Vidimo da sve kolone koje imaju mešovite podatke jesu
 one koje predstavljaju trajanje u mesecima. Konvertujemo podatke kao "Od 2010" u broj meseci
 poređenjem sa datumom elektrokonverzije.

In [79]:
kolone_sa_mesanim_podacima = relevant_data.columns[np.where(relevant_data.dtypes == 'O')]
kolone_sa_mesanim_podacima
# Sve 3 kolone podrazumevaju broj meseci


Index(['ukupno trajanje AF (kada je prvi put dijagnostikovana) u mesecima',
       'trajanje ove epizode AF u mesecima',
       'Vreme u mesecima od ugradnje aparata do elektrokonverzije'],
      dtype='object')

In [80]:
from datetime import datetime
import re

def izdvoj_mesece_iz_niske(x, datum):
    od_search = re.search('[oO]d[a-z ]*(\d{4})', x.lower()) # na jednom mestu piše "0d2010" sa nulom
    if od_search:
        godina = od_search.group(1)
        return (datum - datetime(int(godina), 1, 1)).days // 30

    god_search = re.search('(\d+)\s*god', x.lower())
    if god_search: return int(god_search.group(1)) * 12

    mesec_search = re.search('(\d+)\s*mesec', x.lower())
    if mesec_search: return int(mesec_search.group(1))

    dan_search = re.search('(\d+)\s*dan', x.lower())
    if dan_search: return int(dan_search.group(1)) // 30

    return -1

# print(izdvoj_mesece_iz_niske('od2002', datetime(2014, 1, 21)))
# print(izdvoj_mesece_iz_niske('od 2004', datetime(2014, 1, 21)))
# print(izdvoj_mesece_iz_niske('5god', datetime(2014, 1, 21)))
# print(izdvoj_mesece_iz_niske('1 godina', datetime(2014, 1, 21)))

In [81]:
pd.options.mode.chained_assignment = None # dobijamo upozorenja zbog relevant_data = data[...]
for index, row in relevant_data.iterrows():
    for col in kolone_sa_mesanim_podacima:
        val = relevant_data.loc[index, col]
        new_val = val
        datum = row['Datum elektrokonverzije']
        if isinstance(val, int):
            continue
        elif isinstance(val, datetime):
            new_val = (datum - val).days // 30
        elif isinstance(val, str):
            new_val = izdvoj_mesece_iz_niske(val, datum)
        else:
            new_val = -1
            # print(val, new_val)
        relevant_data.loc[index, col] = new_val

Na kraju izbacujemo datum jer nam više ne treba, a nije relevantan za klasifikaciju.

In [82]:
relevant_data.drop(columns=['Datum elektrokonverzije'], inplace=True)

Uklanjamo atribute kojima su sve vrednosti 0 jer ne utiču na klasifikaciju.

In [83]:
columns_to_remove=[]
for column in relevant_data.columns:
    if(list(relevant_data[column]).count(0) == relevant_data.shape[0]):
        columns_to_remove.append(column)
columns_to_remove

['Digoksin.1']

In [84]:
relevant_data = relevant_data.drop(columns_to_remove, axis=1)

Proveravamo kolone sa NaN. Ručnom analizom procenjujemo da možemo da ih izbacimo ili popunimo nulama.

In [85]:
# Kolone sa NaN
print('\n'.join(relevant_data.columns[np.where(relevant_data.isna().any())]))

# ESD LK                                                        TODO (1 red)
# NYHA klasa (ukoliko ima HF)                                   Popuniti nulama jer nema HF
# vrednost INR 2-3                                              Izbacujemo zbog korelacije sa podatkom OAKT=2 (19 redova)
# Implantiran CIED                                              Popuniti nulama (2 reda)
# NEPOSREDNE KOMPLIKACIJE U VEZI SA ELEKTROKONVERZIJOM          Popuniti nulama (1 red)
# Održavanje sinusnog ritma nakon godinu dana                   Popuniti nulama

relevant_data.drop(columns=['vrednost INR 2-3'], inplace=True)
relevant_data.fillna(0, inplace=True)
# relevant_data

ESD LK
NYHA klasa (ukoliko ima HF)
vrednost INR 2-3
Implantiran CIED
NEPOSREDNE KOMPLIKACIJE U VEZI SA ELEKTROKONVERZIJOM
Održavanje sinusnog ritma nakon godinu dana


In [86]:
relevant_data.to_csv('elektrokonverzija_obradjeno.csv')